# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os
import sys
import datetime
import numpy as np
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization, \
                                            Add, Input, Conv1D, MaxPooling1D, Flatten, \
                                            Lambda

from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.metrics import classification_report

In [3]:
os.chdir("/content/drive/MyDrive/Projects/AlarmWaterClassification")

# Utilies

In [4]:
def print_M(conf_M, class_names):
    s = "activity," + ",".join(class_names)
    print(s)
    for i, row in enumerate(conf_M):
        print(class_names[i] + "," + ",".join(map(str, row)))

def print_M_P(conf_M, class_names):
    s = "activity," + ",".join(class_names)
    print(s)
    for i, row in enumerate(conf_M):
        total = sum(row)
        percentages = [str(round(val / total, 2)) if total > 0 else '0' for val in row]
        print(class_names[i] + "," + ",".join(percentages))

def showResult(result, y_test, class_names):
    predictions = [np.argmax(y) for y in result]
    expected = [np.argmax(y) for y in y_test]

    num_labels = y_test.shape[1]
    conf_M = [[0] * num_labels for _ in range(num_labels)]

    for e, p in zip(expected, predictions):
        conf_M[e][p] += 1

    print_M(conf_M, class_names)
    print_M_P(conf_M, class_names)

def load_weight(path):
    model = load_model(path)
    print(model.summary())
    return model

In [9]:
def build_improved_model(input_shape, num_labels):
    inputs = Input(shape=(input_shape, 1))  # Add a channel dimension for Conv1D

    # First convolutional block
    x = Conv1D(32, kernel_size=3, padding='same', kernel_regularizer=l2(0.001))(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.5)(x)

    # Skip connection 1
    skip1 = x

    # Second convolutional block
    x = Conv1D(64, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.5)(x)

    # Ensuring skip1 and x have the same dimensions
    skip1 = Lambda(lambda z: K.resize_images(z, 1, 2, data_format='channels_last'))(skip1)
    x = Add()([x, skip1])

    # Third convolutional block
    x = Conv1D(128, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.5)(x)

    # Skip connection 2
    skip2 = x

    # Fourth convolutional block
    x = Conv1D(256, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.5)(x)

    # Ensuring skip2 and x have the same dimensions
    skip2 = Lambda(lambda z: K.resize_images(z, 1, 2, data_format='channels_last'))(skip2)
    x = Add()([x, skip2])

    # Flatten the output
    x = Flatten()(x)

    # Fully connected layers
    x = Dense(512, kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)

    x = Dense(256)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)

    x = Dense(128)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)

    # Output layer
    outputs = Dense(num_labels, activation='softmax')(x)

    # Create the model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

    return model



In [17]:
def build_alexnet_1d(input_shape, num_labels):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(input_shape, 1)),
        tf.keras.layers.Conv1D(96, 11, strides=4, activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=3, strides=2),
        tf.keras.layers.Conv1D(256, 5, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=3, strides=2),
        tf.keras.layers.Conv1D(384, 3, padding='same', activation='relu'),
        tf.keras.layers.Conv1D(384, 3, padding='same', activation='relu'),
        tf.keras.layers.Conv1D(256, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=3, strides=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_labels, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

# Data

In [5]:
model_weight_out = os.path.join('weights', 'model_16k_1v2.weights.h5')

# if os.path.exists(model_weight_out):
#     sys.exit(f"The same file name exists already: {model_weight_out}")

############### Loading the datasets #####################

print('\nLoading the data\n')

featuresPath = "features_16k/"

class_names = np.load(os.path.join(featuresPath, 'class_names.npy'))

X_train = np.load(os.path.join(featuresPath, 'X_train.npy'))
y_train = np.load(os.path.join(featuresPath, 'y_train.npy'))

X_val = np.load(os.path.join(featuresPath, 'X_val.npy'))
y_val = np.load(os.path.join(featuresPath, 'y_val.npy'))

X_test = np.load(os.path.join(featuresPath, 'X_test.npy'))
y_test = np.load(os.path.join(featuresPath, 'y_test.npy'))

num_labels = y_train.shape[1]


Loading the data



# Data Balancing

In [23]:
print("\nBalancing the data\n")

print("Train Class distribution before balancing:", Counter(np.argmax(y_train, axis=1)))

# Upsampling using SMOTE
smote = SMOTE(sampling_strategy={1: 10000, 2: 9000})
oversampled_features, oversampled_labels = smote.fit_resample(X_train, y_train)

# Downsampling using RandomUnderSampler
undersampler = RandomUnderSampler(sampling_strategy={0: 7300})
undersampled_features, undersampled_labels = undersampler.fit_resample(
    oversampled_features, oversampled_labels)

print("Train Class distribution after balancing:", Counter(
    np.argmax(undersampled_labels, axis=1)))

X_train = undersampled_features
y_train = undersampled_labels


Balancing the data

Train Class distribution before balancing: Counter({1: 9000, 2: 8000, 0: 7300})


/usr/local/lib/python3.10/dist-packages/imblearn/utils/_validation.py:313: UserWarning: After over-sampling, the number of samples (10000) in class 1 will be larger than the number of samples in the majority class (class #1 -> 9000)
  warnings.warn(


Train Class distribution after balancing: Counter({1: 10000, 2: 9000, 0: 7300})


# Training

In [24]:
###################### Training the model ###########################3
print("\nTraining the model\n")

model = build_improved_model(X_train.shape[1], num_labels)
# model.summary()

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return float(lr * tf.math.exp(-0.1))

callback = LearningRateScheduler(scheduler)

model.fit(X_train, y_train, batch_size=32, epochs=10,
          validation_data=(X_val, y_val), callbacks=[callback])


Training the model

Epoch 1/10
822/822 [==============================] - 7s 7ms/step - loss: 0.4557 - accuracy: 0.8229 - val_loss: 58.4820 - val_accuracy: 0.6422 - lr: 0.0010
Epoch 2/10
822/822 [==============================] - 4s 5ms/step - loss: 0.2591 - accuracy: 0.9017 - val_loss: 77.6976 - val_accuracy: 0.6487 - lr: 0.0010
Epoch 3/10
822/822 [==============================] - 4s 5ms/step - loss: 0.2052 - accuracy: 0.9187 - val_loss: 82.5194 - val_accuracy: 0.6407 - lr: 0.0010
Epoch 4/10
822/822 [==============================] - 5s 6ms/step - loss: 0.1738 - accuracy: 0.9314 - val_loss: 91.5835 - val_accuracy: 0.6573 - lr: 0.0010
Epoch 5/10
822/822 [==============================] - 4s 5ms/step - loss: 0.1492 - accuracy: 0.9405 - val_loss: 119.5494 - val_accuracy: 0.6465 - lr: 0.0010
Epoch 6/10
822/822 [==============================] - 4s 5ms/step - loss: 0.1307 - accuracy: 0.9473 - val_loss: 127.2583 - val_accuracy: 0.6451 - lr: 0.0010
Epoch 7/10
822/822 [=====================

# Testing

In [25]:
print("\nTesting the model\n")

result = model.predict(X_test)

cnt, cnt_alarm, cnt_other, cnt_water = 0, 0, 0, 0
alarm_num, other_num, water_num = (sum(np.argmax(y_test, axis=1) == 0),
                                    sum(np.argmax(y_test, axis=1) == 1),
                                    sum(np.argmax(y_test, axis=1) == 2))

for i in range(len(y_test)):
    pred = np.argmax(result[i])
    if np.argmax(y_test[i]) == pred:
        cnt += 1
        if pred == 0:
            cnt_alarm += 1
        elif pred == 1:
            cnt_other += 1
        else:
            cnt_water += 1

acc = round(cnt * 100 / len(y_test), 2)
acc_alarm = round(cnt_alarm * 100 / alarm_num, 2)
acc_other = round(cnt_other * 100 / other_num, 2)
acc_water = round(cnt_water * 100 / water_num, 2)

print(
    f"Total Accuracy: {acc}%, Alarm Accuracy: {acc_alarm}%, Others Accuracy: {acc_other}%, Water Accuracy: {acc_water}%")

showResult(result, y_test, class_names)

print("\n")
print(classification_report(
    np.argmax(y_test, axis=1),
    np.argmax(result, axis=1),
    target_names=list(class_names)
))


Testing the model

23/23 [==============================] - 0s 3ms/step
Total Accuracy: 85.97%, Alarm Accuracy: 91.08%, Others Accuracy: 79.44%, Water Accuracy: 73.91%
activity,Alarm,Water,Other
Alarm,388,36,2
Water,41,197,10
Other,3,9,34
activity,Alarm,Water,Other
Alarm,0.91,0.08,0.0
Water,0.17,0.79,0.04
Other,0.07,0.2,0.74


              precision    recall  f1-score   support

       Alarm       0.90      0.91      0.90       426
       Water       0.81      0.79      0.80       248
       Other       0.74      0.74      0.74        46

    accuracy                           0.86       720
   macro avg       0.82      0.81      0.82       720
weighted avg       0.86      0.86      0.86       720

